In [13]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    Show and calculate the mean idle power consumption.
    It takes the measurements with active_cores equal to zero and computes the mean.
'''

datadir = "/datasets/eurora_data/db1/"
infile_nodes = datadir + "nodes.csv"
outfile_nodes = datadir + "nodes.csv"

# settings
interval_comment = "WholeData"
suffix = "_1min_"

## notice that 43 doesn't exists in the db
nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
#nodes=['01'] # test

In [14]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)

# Visualize

In [18]:
nodes_data

,node_id,cpu_type,core_idle
0,1,2_1_ghz,3.520750
1,2,2_1_ghz,3.299485
2,3,2_1_ghz,7.900296
3,4,2_1_ghz,8.584291
4,5,2_1_ghz,9.435085
5,6,2_1_ghz,8.903622
6,7,2_1_ghz,8.927407
7,8,2_1_ghz,7.693259
8,9,2_1_ghz,7.352493
9,10,2_1_ghz,7.940631


# Calculate (distruptive)

In [16]:
nodes_data['core_idle'] = 0.0
#nodes_data

In [17]:
for node in nodes:
    print(node)

    # read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment + "_active_cores_and_jobs"
    node_data = pd.read_csv(infile_node + ".csv")
    
    node_data = node_data.dropna()
    idle_node_data = node_data[node_data['active_cores'] == 0]
    idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] > 10]
    #idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] < 50]
    print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
    print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))
    print("cpu0 min value: {}, cpu1 min value: {}".format(idle_node_data['pow_tot_0'].min(), idle_node_data['pow_tot_1'].min()))
    
    nodes_data.at[int(node)-1, 'core_idle'] = (idle_node_data['pow_tot_0'].mean() + idle_node_data['pow_tot_1'].mean()) / 16
    #nodes_data['core_idle'] = (idle_node_data['pow_tot_0'].mean() + idle_node_data['pow_tot_1'].mean()) / 16

01
252392/271241
cpu0 mean value: 28.112618894407078, cpu1 mean value: 28.219376849154415
cpu0 min value: 10.565370648150001, cpu1 min value: 10.71721006915
02
60597/64147
cpu0 mean value: 26.800771034534492, cpu1 mean value: 25.99099194383926
cpu0 min value: 21.290223736088887, cpu1 min value: 21.09598189897778
03
146210/265056
cpu0 mean value: 63.524664856582405, cpu1 mean value: 62.88006687905348
cpu0 min value: 12.100336670999999, cpu1 min value: 11.86998460425
04
135067/258529
cpu0 mean value: 68.17592045810277, cpu1 mean value: 69.17274345060748
cpu0 min value: 21.2232785734, cpu1 min value: 21.202846437466672
05
20912/62986
cpu0 mean value: 74.2580753767135, cpu1 mean value: 76.70327773859486
cpu0 min value: 23.16602243978333, cpu1 min value: 23.812989029375
06
68158/162815
cpu0 mean value: 72.6319791916731, cpu1 mean value: 69.82597005154348
cpu0 min value: 22.489340489633328, cpu1 min value: 21.158450549033336
07
139717/263426
cpu0 mean value: 71.041053275493, cpu1 mean value:

115397/247679
cpu0 mean value: 144.63776506159513, cpu1 mean value: 138.22321785501538
cpu0 min value: 12.8377393707, cpu1 min value: 13.162967594349999
56
142177/247400
cpu0 mean value: 108.12876705432248, cpu1 mean value: 109.16760116566104
cpu0 min value: 20.372713386775, cpu1 min value: 19.469324549275
57
155139/246732
cpu0 mean value: 101.49805304030976, cpu1 mean value: 103.71884256316156
cpu0 min value: 21.030450456649998, cpu1 min value: 19.725038091450006
58
154025/247731
cpu0 mean value: 114.78525353255935, cpu1 mean value: 116.2655171619009
cpu0 min value: 19.90362479885, cpu1 min value: 18.86896637425
59
155074/247754
cpu0 mean value: 102.87090694630902, cpu1 mean value: 107.84527449209162
cpu0 min value: 19.5102764204, cpu1 min value: 19.9689541829
60
130031/224292
cpu0 mean value: 104.00041981291861, cpu1 mean value: 107.50162234942776
cpu0 min value: 18.0582787858, cpu1 min value: 17.681887314233332
61
161360/247898
cpu0 mean value: 114.9161418688683, cpu1 mean value: 11

In [10]:
nodes_data

,node_id,cpu_type,core_idle
0,1,2_1_ghz,3.520750
1,2,2_1_ghz,3.299485
2,3,2_1_ghz,7.900296
3,4,2_1_ghz,8.584291
4,5,2_1_ghz,9.435085
5,6,2_1_ghz,8.903622
6,7,2_1_ghz,8.927407
7,8,2_1_ghz,7.693259
8,9,2_1_ghz,7.352493
9,10,2_1_ghz,7.940631


## Write to file

In [57]:
nodes_data.to_csv(outfile_nodes + ".csv")

# DEBUG

In [44]:
for node in nodes:
    print(node)

    # read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
    infile_node = datadir + "CPUs/" + interval_comment + "/node" + node + suffix + interval_comment + "_active_cores_and_jobs"
    node_data = pd.read_csv(infile_node + ".csv")
    
    idle_node_data = node_data[node_data['pow_tot_1'] < 0.0]
    print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
    #print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))
    #print("cpu0 min value: {}, cpu1 min value: {}".format(idle_node_data['pow_tot_0'].min(), idle_node_data['pow_tot_1'].min()))

01
0/718561
02
0/718561
03
0/718561
04
0/718561
05
0/718561
06
0/718561
07
0/718561
08
0/718561
09
0/718561
10
0/718561
11
0/718561
12
0/718561
13
0/718561
14
0/718561
15
0/718561
16
0/718561
17
0/718561
18
0/718561
19
0/718561
20
0/718561
21
0/718561
22
0/718561
23
0/718561
24
0/718561
25
0/718561
26
0/718561
27
0/718561
28
0/718561
29
0/718561
30
0/718561
31
0/718561
32
0/718561
33
0/718561
34
0/718561
35
0/718561
36
0/718561
37
0/718561
38
0/718561
39
0/718561
40
0/718561
41
0/718561
42
0/718561
44
0/718561
45
0/718561
46
0/718561
47
0/718561
48
0/718561
49
0/718561
50
0/718561
51
0/718561
52
0/718561
53
0/718561
54
0/718561
55
0/718561
56
0/718561
57
0/718561
58
0/718561
59
0/718561
60
0/718561
61
0/718561
62
0/718561
63
0/718561
64
0/718561


In [36]:
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + nodes[0] + suffix + interval_comment + "_active_cores_and_jobs"
node_data = pd.read_csv(infile_node + ".csv")

idle_node_data = node_data[node_data['active_cores'] == 0]
idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] < 50]
print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))

243534/718561
cpu0 mean value: 25.813079300855634, cpu1 mean value: 26.01664736905744


In [57]:
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "05" + suffix + interval_comment + "_active_cores_and_jobs"
node_data = pd.read_csv(infile_node + ".csv")


print(node_data['pow_tot_1'].min())
#idle_node_data = node_data[node_data['active_cores'] == 0]
idle_node_data = node_data[node_data['pow_tot_1'] < 1]
idle_node_data
#print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
#print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))

6.723500000000003e-06


,Unnamed: 0,timestamp,pow_tot_0,pow_tot_1,active_cores,active_jobs,active_gpus,active_mics
2225,2225,2014-04-01 13:05:00,0.000015,0.000007,0,0,0,0


In [34]:
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + nodes[45] + suffix + interval_comment + "_active_cores_and_jobs"
node_data = pd.read_csv(infile_node + ".csv")

idle_node_data = node_data[node_data['active_cores'] == 0]
idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] < 50]
print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))

63111/718561
cpu0 mean value: 28.211773935220048, cpu1 mean value: 32.71093907957488


In [33]:
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + nodes[46] + suffix + interval_comment + "_active_cores_and_jobs"
node_data = pd.read_csv(infile_node + ".csv")

idle_node_data = node_data.dropna()
idle_node_data = idle_node_data[idle_node_data['active_cores'] == 0]
idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] < 50]
print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))

66065/718561
cpu0 mean value: 28.120741339353966, cpu1 mean value: 29.311423272710826


In [32]:
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + nodes[60] + suffix + interval_comment + "_active_cores_and_jobs"
node_data = pd.read_csv(infile_node + ".csv")

idle_node_data = node_data[node_data['active_cores'] == 0]
idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] < 50]
print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))

75719/718561
cpu0 mean value: 27.479039640905327, cpu1 mean value: 29.652909707022253


In [31]:
# read the power mesuraments file obtained with "keep_nodes_data_by_period_1min_sample.py"
infile_node = datadir + "CPUs/" + interval_comment + "/node" + nodes[5] + suffix + interval_comment + "_active_cores_and_jobs"
node_data = pd.read_csv(infile_node + ".csv")

idle_node_data = node_data[node_data['active_cores'] == 0]
idle_node_data = idle_node_data[idle_node_data['pow_tot_0'] < 50]
print("{}/{}".format(idle_node_data.shape[0], node_data.shape[0]))
print("cpu0 mean value: {}, cpu1 mean value: {}".format(idle_node_data['pow_tot_0'].mean(), idle_node_data['pow_tot_1'].mean()))

24842/718561
cpu0 mean value: 28.877879310071624, cpu1 mean value: 26.664967983472614
